In [44]:
# Importing libraries
from keras.utils.np_utils import to_categorical

import pandas as pd
import numpy as np

import re

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import combinations
from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import tensorflow as tf
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import numpy as np

import matplotlib.pyplot as plt
import nltk

nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

nltk.download('punkt')

import spacy 
from spacy import displacy

import random

# from rake_nltk import Rake

from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense

import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector

from gensim.scripts.glove2word2vec import glove2word2vec

# import en_core_web_lg
# nlp = en_core_web_lg.load()

from nltk.tokenize import word_tokenize

import gensim
from scipy import spatial

import csv
import tensorflow as tf

# import transformers
# transformers.logging.set_verbosity(transformers.logging.CRITICAL)

# from sentence_transformers import SentenceTransformer, util
# import nmslib

# distilbert = SentenceTransformer('distilbert-base-uncased')

import multiprocessing
import math

import time
from gensim.models import KeyedVectors
from gensim.models.fasttext import FastText
import pandas as pd
import gensim
gensim.__version__

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import words

from keras.models import *
from keras.layers import *
from keras.callbacks import *

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [50]:
#importing dataset
data = pd.read_csv("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/ml-25m/movie_all_info.csv")
movtag4 = pd.read_pickle("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/Final datasets/2_ner_tags.pkl")

In [51]:
movtag4.head(2)

,movieId,combined_tags_nodup,tag_people,tag_org,tag_norp,tag_location,tags
0,1,"[Owned, imdb, top, Pixar, time, travel, childr...","[chindren pixar, Tom Hanks, Woody, Tim Allen, ...","[imdb, Pixar, Disney, National Film Registry, ...",[],[],"[top, time, travel, comedy, funny, witty, anim..."
1,2,"[Robin, Williams, time, travel, fantasy, based...","[Robin Williams, Chris Van Allsburg, Kirsten D...",[Zathura],[],[],"[Robin, time, travel, fantasy, based, book, bo..."


In [52]:
embeddings_index = dict()
f = open('/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/glove.6B.100d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [56]:
# from keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from nltk.corpus import words

def Glove_embeddings(data, ratings_dataset, colname):
  #creating the vocabulary
  tokenizer = Tokenizer()
  data[colname] = data[colname].apply(lambda x: ' '.join(x))
  x_tr = data[colname]
  tokenizer.fit_on_texts(x_tr)
  x_tr_seq  = tokenizer.texts_to_sequences(x_tr)
  x_tr_seq  = pad_sequences(x_tr_seq, maxlen=100)
  size_of_vocabulary=len(tokenizer.word_index) + 1

  #Generating the embedding Matrix
  embedding_matrix = np.zeros((size_of_vocabulary, 100))

  for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector
  
  #Building the sequential model
  model=Sequential()
  model.add(Embedding(size_of_vocabulary,100,weights=[embedding_matrix],input_length=100,trainable=False)) 
  model.add(LSTM(128,return_sequences=True,dropout=0.2))
  model.add(GlobalMaxPooling1D())
  model.add(Dense(64,activation='relu')) 
  model.add(Dense(10,activation='sigmoid')) 
  model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=["acc"]) 
  es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)  
  mc=ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)  
  print(model.summary())

  #Creating labels to train the NN
  y_tr = ratings_dataset['averageRating_imdb']
  y_tr = np.array(y_tr)
  y_tr[np.where(np.isnan(y_tr))] = 0
  y_tr = to_categorical(y_tr)

  #Fitting the model
  history = model.fit(np.array(x_tr_seq),np.array(y_tr),batch_size=128,epochs=1)

  #Extracting the GloVe Embeddings and appending to the main dataframe.
  embeddings = model.layers[0].get_weights()[0]
  embeddings.shape
  data[colname] = data[colname].apply(lambda x: word_tokenize(x))
  glove_embed = []
  count = 0
  for i in data[colname]:
    lst = []
    count = count + 1
    for j in range(len(i)):
      try:
        lst.append(embeddings[tokenizer.word_index[i[j]]])
      except:
        pass
    emb = np.array(lst)
    glove_embed.append(np.sum(emb, axis = 0)/len(emb))

  data['glove_' + colname] = glove_embed

In [57]:
# Creating GloVe Embeddings for tag_people
Glove_embeddings(movtag4, data, 'tag_people')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 100)          1547000   
                                                                 
 lstm_3 (LSTM)               (None, 100, 128)          117248    
                                                                 
 global_max_pooling1d_3 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 10)                650       
                                                                 
Total params: 1,673,154
Trainable params: 126,154
Non-trainable params: 1,547,000
______________________________________

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [66]:
# Creating GloVe Embeddings for tag_org
Glove_embeddings(movtag4, data, 'tag_org')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 100, 100)          575600    
                                                                 
 lstm_4 (LSTM)               (None, 100, 128)          117248    
                                                                 
 global_max_pooling1d_4 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dense_9 (Dense)             (None, 10)                650       
                                                                 
Total params: 701,754
Trainable params: 126,154
Non-trainable params: 575,600
__________________________________________

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [67]:
# Creating GloVe Embeddings for tag_norp
Glove_embeddings(movtag4, data, 'tag_norp')

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 100)          71400     
                                                                 
 lstm_5 (LSTM)               (None, 100, 128)          117248    
                                                                 
 global_max_pooling1d_5 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dense_11 (Dense)            (None, 10)                650       
                                                                 
Total params: 197,554
Trainable params: 126,154
Non-trainable params: 71,400
___________________________________________

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [68]:
# Creating GloVe Embeddings for tag_location
Glove_embeddings(movtag4, data, 'tag_location')

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 100, 100)          173100    
                                                                 
 lstm_6 (LSTM)               (None, 100, 128)          117248    
                                                                 
 global_max_pooling1d_6 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_12 (Dense)            (None, 64)                8256      
                                                                 
 dense_13 (Dense)            (None, 10)                650       
                                                                 
Total params: 299,254
Trainable params: 126,154
Non-trainable params: 173,100
__________________________________________

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [69]:
# Creating GloVe Embeddings for tags
Glove_embeddings(movtag4, data, 'tags')

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_7 (Embedding)     (None, 100, 100)          1304200   
                                                                 
 lstm_7 (LSTM)               (None, 100, 128)          117248    
                                                                 
 global_max_pooling1d_7 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_14 (Dense)            (None, 64)                8256      
                                                                 
 dense_15 (Dense)            (None, 10)                650       
                                                                 
Total params: 1,430,354
Trainable params: 126,154
Non-trainable params: 1,304,200
______________________________________

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountere

In [70]:
movtag4.to_pickle("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/Final datasets/5_Glove_tags.pkl")

In [71]:
movtag4.head()

,movieId,combined_tags_nodup,tag_people,tag_org,tag_norp,tag_location,tags,glove_tag_people,glove_tag_org,glove_tag_norp,glove_tag_location,glove_tags
0,1,"[Owned, imdb, top, Pixar, time, travel, childr...","[chindren, pixar, Tom, Hanks, Woody, Tim, Alle...","[imdb, Pixar, Disney, National, Film, Registry...",[],[],"[top, time, travel, comedy, funny, witty, anim...","[0.112047575, -0.19083759, 0.16281143, -0.8592...","[0.18163931, -0.24781334, 0.255729, -0.0454656...",NaN,NaN,"[0.04372807, 0.1893854, 0.28725392, -0.2534521..."
1,2,"[Robin, Williams, time, travel, fantasy, based...","[Robin, Williams, Chris, Van, Allsburg, Kirste...",[Zathura],[],[],"[Robin, time, travel, fantasy, based, book, bo...",NaN,NaN,NaN,NaN,"[-0.032221712, 0.13218969, 0.42905343, -0.3109..."
2,3,"[funny, best, friend, duringcreditsstinger, fi...","[Jack, Lemmon, Walter, Matthau, grun, Howard, ...","[comedinha, de, velhinhos, engraÃƒÂ§ada, old]",[],[],"[funny, best, friend, fishing, old, man, seque...","[0.030375, -0.22178, 0.06272, 0.081105, 0.0518...","[0.1700775, -0.25163, -0.1419, 0.095952496, -0...",NaN,NaN,"[0.2154257, 0.010377906, 0.270296, -0.3554276,..."
3,4,"[based, novel, book, chick, flick, divorce, in...",[CLV],[],[],[],"[based, novel, book, chick, flick, divorce, in...",NaN,NaN,NaN,NaN,"[0.18884882, -0.0002479418, -0.008310458, -0.2..."
4,5,"[aging, baby, confidence, contraception, daugh...","[steve, martin, Comedy, Touching, CLV, Diane, ...",[],[],[],"[aging, baby, confidence, contraception, daugh...","[-0.42567998, -0.57784, -0.12303001, -0.96325,...",NaN,NaN,NaN,"[0.26442227, 0.12181655, 0.23594351, -0.031013..."
